In [45]:
from JQLBuilder import JQLBuilder
from Enums import Enums
from JiraFacade import JiraFacade
from typing import Optional

In [46]:
developed_statuses = ['Ready To Test', 'Testing', 'Test Passed', 'Ready To Deploy', 'Deployed', 'Done']
not_started_statuses = ['Sprint Backlog']
in_progress_statuses = ['In Progress', 'Code Review', 'Revision']
invalid_statuses = ['Analysis', 'Product Backlog', 'Blocked', 'Closed']

In [47]:
def get_developer(assigned_to: str, status: str, developed_by: Optional[str]):
    if developed_by:
        return developed_by

    if status in not_started_statuses + in_progress_statuses:
        return assigned_to

    return ''

def get_state(status: str):
    if status in developed_statuses:
        return 'developed'
    if status in not_started_statuses:
        return 'not_started'
    if status in in_progress_statuses:
        return 'in_progress'
    if status in invalid_statuses:
        return 'invalid_ticket'
    return ''


In [48]:
jql = JQLBuilder().set_project(Enums.project_shopping).set_sprints_from_squads(
    squad_names=[Enums.squad_name_discovery, Enums.squad_name_omc],
    sprint='04', quarter='3').get()

In [49]:
df = JiraFacade().get_df_from_jql(jql, 100)

In [50]:
df['developer'] = df.apply(lambda x: get_developer(x['assigned_to'], x['status'], x['developed_by']), axis=1)
df['state'] = df.apply(lambda x: get_state(x['status']), axis=1)

In [ ]:
df.groupby(['squad_name', 'team_name', 'developer', 'state']).agg(
    {'key': 'count', 'estimate': 'sum'}).reset_index().rename(
    columns={'key': 'number_of_tickets', 'estimate': 'story_point'}).pivot(
    index=['team_name', 'squad_name', 'developer'], columns=['state'],
    values=['story_point', 'number_of_tickets']).fillna(0)